# PANDAS BASE V: CALIDAD DE DATOS: CORRECIÓN

En este módulo aprenderemos a corregir los problemas de calidad de datos que habremos detectado aplicando las técnicas del módulo anterior.

Concretamente aprenderemos a:

* Convertir los tipos de las variables
* Eliminar ciertas variables o registros
* Insertar nuevas variables
* Qué hacer con los nulos
* Eliminar los duplicados
* Renonmbrar variables e índices
* Reemplazar valores por otros
* Recodificar variables
* Corregir calidad de variables de tipo texto

**Carga de paquetes**

In [2]:
import pandas as pd

**Importación de datos**

In [2]:
df = pd.read_csv('../../00_Datasets//DataSetKivaCreditScoring.csv', 
                 sep = ';', index_col = 'id',
                 parse_dates = ['Funded Date','Paid Date'])
df.head(2)

,Funded Date,Funded Amount,Country,Country Code,Loan Amount,Paid Date,Paid Amount,Activity,Sector,Delinquent,Name,Use,Status
id,,,,,,,,,,,,,
84,2005-03-31 06:27:55+00:00,500,Uganda,UG,500,2005-12-13 12:00:40+00:00,500.0,Butcher Shop,Food,False,Justine,"Buy bulls, open a butcher shop",paid
85,2005-03-31 06:27:55+00:00,500,Uganda,UG,500,2005-12-13 12:04:33+00:00,500.0,Food Production/Sales,Food,False,Geoffrey,Buying more produce each time for greater profit,paid


## CONVERTIR TIPOS DE VARIABLE

Tenemos dos formas para convertir tipos de variables:

* con el método general astype() y especificando el tipo deseado
* con métodos concretos para ciertos tipos de datos

astype nos da más flexibilidad pero funcionará peor en casos más complejos.

### CONVERTIR TIPOS CON ASTYPE

astype()

Convierte al tipo especificado.

Notar que no convierte inplace.

https://pandas.pydata.org/docs/reference/api/pandas.Series.astype.html

Ejemplos de tipos los que podemos convertir:

* int
* float
* category
* O
* ...

In [3]:
#Revisamos los tipos actuales
df.dtypes

Funded Date      datetime64[ns, UTC]
Funded Amount                  int64
Country                       object
Country Code                  object
Loan Amount                    int64
Paid Date        datetime64[ns, UTC]
Paid Amount                  float64
Activity                      object
Sector                        object
Delinquent                      bool
Name                          object
Use                           object
Status                        object
dtype: object

In [4]:
#Convertimos country a categórica
df.Country.astype('category').dtype

CategoricalDtype(categories=['Afghanistan', 'Azerbaijan', 'Bulgaria', 'Cambodia',
                  'Cameroon', 'Dominican Republic', 'Ecuador', 'Gaza', 'Ghana',
                  'Honduras', 'India', 'Indonesia', 'Kenya', 'Mexico',
                  'Moldova', 'Mozambique', 'Nicaragua', 'Nigeria', 'Samoa',
                  'Senegal', 'Tanzania',
                  'The Democratic Republic of the Congo', 'Togo', 'Uganda',
                  'Ukraine'],
, ordered=False, categories_dtype=object)

Podemos hacer varias conversiones simultáneas pasándolo como un diccionario.

In [5]:
tipos = {'Funded Amount':'float',
         'Country Code':'category',
         'Delinquent':'int'}
df.astype(tipos).dtypes

Funded Date      datetime64[ns, UTC]
Funded Amount                float64
Country                       object
Country Code                category
Loan Amount                    int64
Paid Date        datetime64[ns, UTC]
Paid Amount                  float64
Activity                      object
Sector                        object
Delinquent                     int64
Name                          object
Use                           object
Status                        object
dtype: object

O convertir por bloques combinándola con select_dtypes()

In [6]:
df.select_dtypes('object').astype('category').dtypes

Country         category
Country Code    category
Activity        category
Sector          category
Name            category
Use             category
Status          category
dtype: object

### CONVERTIR TIPOS CON FUNCIONES CONCRETAS

#### Convertir a tipo numérico

**to_numeric()**

https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html

Parámetros más importantes:

* errors: personaliza el comportamiento cuando no pueda convertir algo. Lo más típico es ponerlo a 'coerce' que forzará un nulo


In [7]:
pd.to_numeric(df['Funded Date']).dtype

dtype('int64')

#### Convertir a tipo fecha

**to_datetime()**

https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

Parámetros más importantes:

* dayfirst: indentifica si el componente inicial de la fecha es el día. Por defecto viene a False
* errors: personaliza el comportamiento cuando no pueda convertir algo. Lo más típico es ponerlo a 'coerce' que forzará un nulo
* format: el formato de la cadena que le estamos pasando. Para los códigos ver: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior


In [8]:
#En muchos casos por defecto lo hará bien automáticamente
pd.to_datetime('13/08/2020', dayfirst = True)

Timestamp('2020-08-13 00:00:00')

In [9]:
#En muchos casos por defecto lo hará bien automáticamente
pd.to_datetime('13-Aug-2020')

Timestamp('2020-08-13 00:00:00')

In [ ]:
#Pero a veces no lo hace bien
pd.to_datetime('13082020')

In [23]:
#Y podemos corregirlo con el format
pd.to_datetime('13082020', format='%d%m%Y')

Timestamp('2020-08-13 00:00:00')

**Cambiar el formato de una fecha**

Cuando ya tenemos una fecha en tipo datetime podemos cambiar su formato con .dt.strftime()

Los códigos de formato son los mismos: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [24]:
#Así es el formato origen de nuestra fecha
df['Funded Date'].head()

id
84   2005-03-31 06:27:55+00:00
85   2005-03-31 06:27:55+00:00
86   2005-03-31 06:27:55+00:00
88   2005-03-31 06:27:55+00:00
89   2005-03-31 06:27:55+00:00
Name: Funded Date, dtype: datetime64[ns, UTC]

In [25]:
#Se lo cambiamos a uno europeo
df['Funded Date'].dt.strftime('%d/%m/%Y')

id
84      31/03/2005
85      31/03/2005
86      31/03/2005
88      31/03/2005
89      31/03/2005
           ...    
5255    27/03/2007
5256    27/03/2007
5257    02/06/2007
5258    24/03/2007
5259    05/06/2007
Name: Funded Date, Length: 5146, dtype: object

#### Convertir a tipo categórica ORDENADA

Las variables categóricas normalmente serán nominales, es decir sin tener un orden.

Pero también es muy frecuente que tengan un orden, siendo entonces ordinales.

Si Pandas ha importado la variable como Object es buena práctica pasarla a categórica, entre otras cosas será mucho más eficiente en tamaño y procesamiento.

Para convertir a nominal podemos usar el astype('category') que ya conocemos.

Para convertir a ordinal y establecer el orden de una variable usamos CategoricalDtype().

El proceso a seguir es:

1. Crear el orden deseado pasándole una lista y el parámetro ordered = True a CategoricalDtype()
2. Pasarle ese orden a astype() como una conversión normal

https://pandas.pydata.org/docs/reference/api/pandas.CategoricalDtype.html


Por ejemplo vamos a convertir Status a una categórica ordenada

In [26]:
#Primero comprobamos el tipo actual
df.Status.dtype

dtype('O')

In [27]:
#Comprobamos los diferentes valores que puede tomar
df.Status.unique()

array(['paid', 'defaulted', 'refunded', nan, 'deleted'], dtype=object)

In [28]:
#Definimos el orden
orden_Status = pd.CategoricalDtype(['deleted','defaulted','paid','refunded'],
                                   ordered = True)

In [29]:
#Aplicamos el orden y comprobamos sus valores
df['Status'] = df['Status'].astype(orden_Status)
df['Status'].unique()

['paid', 'defaulted', 'refunded', NaN, 'deleted']
Categories (4, object): ['deleted' < 'defaulted' < 'paid' < 'refunded']

In [30]:
#Y vemos que ahora ya su tipo es categórico ordenado
df['Status'].dtype

CategoricalDtype(categories=['deleted', 'defaulted', 'paid', 'refunded'], ordered=True, categories_dtype=object)

Si la variable ya fuera categórica, pero sin orden, o incluso si ya tiene orden pero queremos cambiarlo. Entonces usaremos .cat.set_categories().

Por ejemplo vamos a cambiar el orden poniendo 'paid' primero.

In [31]:
df.Status.cat.set_categories(['paid','deleted','defaulted','refunded'], 
                             ordered = True) 

id
84      paid
85      paid
86      paid
88      paid
89      paid
        ... 
5255    paid
5256    paid
5257    paid
5258    paid
5259    paid
Name: Status, Length: 5146, dtype: category
Categories (4, object): ['paid' < 'deleted' < 'defaulted' < 'refunded']

Repaso de lo aprendido:

- Usa astype() para conversiones generales entre tipos sencillos (numéricos, categóricos).
- Convierte a numérico con pd.to_numeric() usando errors='coerce' para manejar valores problemáticos.
- Transforma textos a fechas con pd.to_datetime(), utilizando 'format' para fechas con estructura específica.
- Usa CategoricalDtype() para variables categóricas ordenadas definiendo claramente su orden lógico.

## ELIMINAR VARIABLES O REGISTROS

**drop()**

Elimina variables o registros.

Notar que por defecto no es inplace.

Drop elimina por nombre, no por posición.

Si quisiéramos eliminar por posición deberíamos obtener primero los nombres de las posiciones a eliminar con .index()

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html

Parámetros más importantes:

* index: para eliminar registros
* columns: para eliminar variables
* inplace: por defecto es False


In [32]:
#Eliminar variables
df.drop(columns = ['Funded Date','Funded Amount']).head(2)

,Pendiente,Country,Country Code,Loan Amount,Paid Date,Paid Amount,Activity,Sector,Delinquent,Name,Use,Status
id,,,,,,,,,,,,
84,0.0,Uganda,UG,500,2005-12-13 12:00:40+00:00,500.0,Butcher Shop,Food,False,Justine,"Buy bulls, open a butcher shop",paid
85,0.0,Uganda,UG,500,2005-12-13 12:04:33+00:00,500.0,Food Production/Sales,Food,False,Geoffrey,Buying more produce each time for greater profit,paid


In [33]:
#Eliminar registros
df.drop(index = [84,85]).head(2)

,Funded Date,Funded Amount,Pendiente,Country,Country Code,Loan Amount,Paid Date,Paid Amount,Activity,Sector,Delinquent,Name,Use,Status
id,,,,,,,,,,,,,,
86,2005-03-31 06:27:55+00:00,500,0.0,Uganda,UG,500,2005-12-13 12:06:56+00:00,500.0,Animal Sales,Agriculture,False,Apollo,Increase number of goats bought and sold each ...,paid
88,2005-03-31 06:27:55+00:00,300,0.0,Uganda,UG,300,2005-08-27 07:33:49+00:00,300.0,Clothing Sales,Clothing,False,Eunice,Buy used clothing (mutumbe) of higher quality ...,paid


In [34]:
#Para eliminar registros por posición, ej los 5 primeros, obtenemos su nombre con index
a_eliminar = df.index[0:5]
a_eliminar

Index([84, 85, 86, 88, 89], dtype='int64', name='id')

In [35]:
#Y después eliminamos
df.drop(index = a_eliminar)

,Funded Date,Funded Amount,Pendiente,Country,Country Code,Loan Amount,Paid Date,Paid Amount,Activity,Sector,Delinquent,Name,Use,Status
id,,,,,,,,,,,,,,
90,2005-03-31 06:27:55+00:00,300,0.0,Uganda,UG,300,2005-08-27 07:38:20+00:00,300.0,Restaurant,Food,False,Christine,"Buy better utensils, plates, and cutleries, an...",paid
91,2005-03-31 06:27:55+00:00,500,0.0,Uganda,UG,500,2005-12-13 12:12:55+00:00,500.0,Fish Selling,Food,False,Elizabeth,"Increase stock, open new location in Tororo to...",paid
95,2005-03-31 06:27:55+00:00,500,0.0,Uganda,UG,500,2006-10-24 05:05:15+00:00,500.0,Cereals,Food,False,Betty,"Buying maize, millet, sorghum, peas, groundnut...",paid
96,2005-03-31 06:27:55+00:00,500,0.0,Uganda,UG,500,2006-10-24 05:02:47+00:00,500.0,Cereals,Food,False,Benna,"Buying millet, sorghum, rice, and other cereal...",paid
97,2005-03-31 06:27:55+00:00,500,0.0,Uganda,UG,500,2006-10-16 06:44:34+00:00,500.0,Bricks,Construction,False,Henry,Increasing retail shop inventory,paid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5255,2007-03-27 05:41:31+00:00,1000,0.0,Ghana,GH,1000,2008-01-10 10:17:02+00:00,1000.0,Construction,Construction,False,Grace,Working capital,paid
5256,2007-03-27 21:58:56+00:00,800,0.0,Ghana,GH,800,2008-01-11 10:16:27+00:00,800.0,Food Market,Food,False,Ysutor,Working capital,paid
5257,2007-06-02 03:57:56+00:00,1000,0.0,Ghana,GH,1000,2008-03-13 10:01:32+00:00,1000.0,Construction,Construction,False,Abla,Working capital,paid


## INSERTAR VARIABLES

**insert()**

Se usa cuando queremos insertar una nueva variable en una posición determinada.

Notar que hace el inplace directamente.

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.insert.html

Parámetros más importantes:

* loc: la posición en la que insertar
* column: nombre de la nueva variable
* value: valores de la variable


In [36]:
#Insertamos nueva variable
df.insert(2,'Pendiente',df['Loan Amount'] - df['Paid Amount'])

ValueError: cannot insert Pendiente, already exists

In [37]:
#Vemos que ya lo ha hecho
df.head(3)

,Funded Date,Funded Amount,Pendiente,Country,Country Code,Loan Amount,Paid Date,Paid Amount,Activity,Sector,Delinquent,Name,Use,Status
id,,,,,,,,,,,,,,
84,2005-03-31 06:27:55+00:00,500,0.0,Uganda,UG,500,2005-12-13 12:00:40+00:00,500.0,Butcher Shop,Food,False,Justine,"Buy bulls, open a butcher shop",paid
85,2005-03-31 06:27:55+00:00,500,0.0,Uganda,UG,500,2005-12-13 12:04:33+00:00,500.0,Food Production/Sales,Food,False,Geoffrey,Buying more produce each time for greater profit,paid
86,2005-03-31 06:27:55+00:00,500,0.0,Uganda,UG,500,2005-12-13 12:06:56+00:00,500.0,Animal Sales,Agriculture,False,Apollo,Increase number of goats bought and sold each ...,paid


Repaso de lo aprendido

- Elimina variables o registros con drop(columns=..., index=...).
- Para eliminar por posición recuerda obtener primero los nombres con index[].
- Usa insert() para añadir una nueva columna directamente en una posición específica (recuerda que esto es siempre inplace).

## ELIMINAR NULOS

**dropna()**

Elimina registros o columnas que tienen nulos.

Notar que por defecto no hace inplace.

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html

Parámetros más importantes:

* axis: Si 0 o index elimina los registros con nulos. Si 1 o columns las variables.
* how: por defecto 'any' que elimina todo el registro o columna simplemente con que haya un nulo. Se puede poner a 'all' que lo eliminará solo si está totalmente a nulos
* subset: si queremos eliminar solo los nulos en algunas variables
* thresh: un umbral para considerar eliminar todo el registro o columna. Ej que haya al menos 100 nulos
* inplace: por defecto False


In [38]:
#Dimensiones del dataset original
df.shape

(5146, 14)

In [39]:
#Dimensiones del dataset tras eliminar las columnas con nulos
df.dropna(axis = 1).shape

(5146, 7)

In [40]:
#Dimensiones del dataset tras eliminar los registros con nulos
df.dropna(axis = 0).shape

(4031, 14)

## REEMPLAZAR NULOS

**fillna()**

Reemplaza los nulos por otro valor.

Se puede reemplazar por un valor fijo como 0, o por un valor calculado al vuelo como la media, o con algún método como copiar el valor superior o inferior.

Notar que por defecto no hace inplace.

https://pandas.pydata.org/docs/reference/api/pandas.Series.fillna.html

Parámetros más importantes:

- value: para reemplazar por un valor fijo o calculado
- inplace: por defecto False,

Otros métodos que son de utilidad:

* ffill() forward-fill para reemplazar por valor superior
* bfill() back-fill para reemplazar por valor inferior


In [41]:
#Conteo de Status incluyendo nulos
df.Status.value_counts(dropna=False)

Status
paid         4031
defaulted     600
refunded      381
deleted        86
NaN            48
Name: count, dtype: int64

In [43]:
#Conteo de Status tras imputar nulos por la moda

moda_Status =df.Status.mode() .values[0]

df.Status.fillna(value = moda_Status).value_counts(dropna=False)

Status
paid         4079
defaulted     600
refunded      381
deleted        86
Name: count, dtype: int64

**CUIDADO:**

Si intentamos reemplazar por un nuevo valor en una variable categórica que no está entre los valores originales tenemos que hacer 2 pasos:

1. Añadir la nueva categoría con .cat.add_categories()
2. Reemplazar el valor

Pandas no añade la nueva categoría directamente.

Por ejemplo, si Country fuera categórica, y supiéramos que sus nulos son de un país que no está entre sus valores actuales, pongamos España, y queremos imputar los nulos por España tendríamos que hacer así:

df.Country.cat.add_categories('España',inplace=True)

df.Country.fillna(value = 'España',inplace=True)

## ELIMINAR DUPLICADOS

**drop_duplicates()**

Elimina registros duplicados. (No tienen en cuenta el índice).

Notar que por defecto no hace inplace.

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html

Parámetros más importantes:

* subset: para definirle las columnas que debe analizar, por defecto son todas
* keep: el registro que quieres que deje, por defecto 'first' pero le puedes poner 'last'
* inplace: por defecto False


In [44]:
df.drop_duplicates()

,Funded Date,Funded Amount,Pendiente,Country,Country Code,Loan Amount,Paid Date,Paid Amount,Activity,Sector,Delinquent,Name,Use,Status
id,,,,,,,,,,,,,,
84,2005-03-31 06:27:55+00:00,500,0.0,Uganda,UG,500,2005-12-13 12:00:40+00:00,500.0,Butcher Shop,Food,False,Justine,"Buy bulls, open a butcher shop",paid
85,2005-03-31 06:27:55+00:00,500,0.0,Uganda,UG,500,2005-12-13 12:04:33+00:00,500.0,Food Production/Sales,Food,False,Geoffrey,Buying more produce each time for greater profit,paid
86,2005-03-31 06:27:55+00:00,500,0.0,Uganda,UG,500,2005-12-13 12:06:56+00:00,500.0,Animal Sales,Agriculture,False,Apollo,Increase number of goats bought and sold each ...,paid
88,2005-03-31 06:27:55+00:00,300,0.0,Uganda,UG,300,2005-08-27 07:33:49+00:00,300.0,Clothing Sales,Clothing,False,Eunice,Buy used clothing (mutumbe) of higher quality ...,paid
89,2005-03-31 06:27:55+00:00,500,0.0,Uganda,UG,500,2005-12-13 12:09:18+00:00,500.0,Food Production/Sales,Food,False,Rose,Buying and reselling of produce,paid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5255,2007-03-27 05:41:31+00:00,1000,0.0,Ghana,GH,1000,2008-01-10 10:17:02+00:00,1000.0,Construction,Construction,False,Grace,Working capital,paid
5256,2007-03-27 21:58:56+00:00,800,0.0,Ghana,GH,800,2008-01-11 10:16:27+00:00,800.0,Food Market,Food,False,Ysutor,Working capital,paid
5257,2007-06-02 03:57:56+00:00,1000,0.0,Ghana,GH,1000,2008-03-13 10:01:32+00:00,1000.0,Construction,Construction,False,Abla,Working capital,paid


Repaso de lo aprendido:

- Usa dropna() para eliminar filas o columnas según criterios específicos de valores nulos (axis, how, subset).
- Aplica fillna() para reemplazar nulos con valores calculados o métodos como forward-fill o back-fill.
- Usa drop_duplicates(subset=..., keep=...) para eliminar registros duplicados y mantener los más relevantes.

## RENOMBRAR VARIABLES O INDICE

**rename()**

Reemplazar los nombres tanto de variables como de los elementos del índice usando un diccionario, con la forma {'viejo_nombre':'nuevo_nombre')

Para renombrar variables lo usaremos con columns y para el índice con index

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html

Parámetros más importantes:

* inplace: por defecto False

In [47]:
df.rename(columns = {'Funded Date':'Fecha_financiacion',
          'Country':'Pais'})

,Fecha_financiacion,Funded Amount,Pendiente,Pais,Country Code,Loan Amount,Paid Date,Paid Amount,Activity,Sector,Delinquent,Name,Use,Status
id,,,,,,,,,,,,,,
84,2005-03-31 06:27:55+00:00,500,0.0,Uganda,UG,500,2005-12-13 12:00:40+00:00,500.0,Butcher Shop,Food,False,Justine,"Buy bulls, open a butcher shop",paid
85,2005-03-31 06:27:55+00:00,500,0.0,Uganda,UG,500,2005-12-13 12:04:33+00:00,500.0,Food Production/Sales,Food,False,Geoffrey,Buying more produce each time for greater profit,paid
86,2005-03-31 06:27:55+00:00,500,0.0,Uganda,UG,500,2005-12-13 12:06:56+00:00,500.0,Animal Sales,Agriculture,False,Apollo,Increase number of goats bought and sold each ...,paid
88,2005-03-31 06:27:55+00:00,300,0.0,Uganda,UG,300,2005-08-27 07:33:49+00:00,300.0,Clothing Sales,Clothing,False,Eunice,Buy used clothing (mutumbe) of higher quality ...,paid
89,2005-03-31 06:27:55+00:00,500,0.0,Uganda,UG,500,2005-12-13 12:09:18+00:00,500.0,Food Production/Sales,Food,False,Rose,Buying and reselling of produce,paid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5255,2007-03-27 05:41:31+00:00,1000,0.0,Ghana,GH,1000,2008-01-10 10:17:02+00:00,1000.0,Construction,Construction,False,Grace,Working capital,paid
5256,2007-03-27 21:58:56+00:00,800,0.0,Ghana,GH,800,2008-01-11 10:16:27+00:00,800.0,Food Market,Food,False,Ysutor,Working capital,paid
5257,2007-06-02 03:57:56+00:00,1000,0.0,Ghana,GH,1000,2008-03-13 10:01:32+00:00,1000.0,Construction,Construction,False,Abla,Working capital,paid


## REEMPLAZAR VALORES

**replace()**

Reemplazar la presencia de un valor por otro en la variable (o en todo el dataframe).

Notar que por defecto no hace inplace.

https://pandas.pydata.org/docs/reference/api/pandas.Series.replace.html

Parámetros más importantes:

* to_replace: el valor viejo a reemplazar. Acepta cosas avanzadas como diccionarios o Regex
* value: el nuevo valor
* inplace: por defecto False

In [48]:
df.Country.replace('Uganda','Uga')

id
84        Uga
85        Uga
86        Uga
88        Uga
89        Uga
        ...  
5255    Ghana
5256    Ghana
5257    Ghana
5258    Ghana
5259    Ghana
Name: Country, Length: 5146, dtype: object

También podemos pasarle un diccionario para reemplazar varios valores a la vez.

In [49]:
valores = {'Uganda':'Uga', 'Ghana':'Gha'}

df.replace(valores)

,Funded Date,Funded Amount,Pendiente,Country,Country Code,Loan Amount,Paid Date,Paid Amount,Activity,Sector,Delinquent,Name,Use,Status
id,,,,,,,,,,,,,,
84,2005-03-31 06:27:55+00:00,500,0.0,Uga,UG,500,2005-12-13 12:00:40+00:00,500.0,Butcher Shop,Food,False,Justine,"Buy bulls, open a butcher shop",paid
85,2005-03-31 06:27:55+00:00,500,0.0,Uga,UG,500,2005-12-13 12:04:33+00:00,500.0,Food Production/Sales,Food,False,Geoffrey,Buying more produce each time for greater profit,paid
86,2005-03-31 06:27:55+00:00,500,0.0,Uga,UG,500,2005-12-13 12:06:56+00:00,500.0,Animal Sales,Agriculture,False,Apollo,Increase number of goats bought and sold each ...,paid
88,2005-03-31 06:27:55+00:00,300,0.0,Uga,UG,300,2005-08-27 07:33:49+00:00,300.0,Clothing Sales,Clothing,False,Eunice,Buy used clothing (mutumbe) of higher quality ...,paid
89,2005-03-31 06:27:55+00:00,500,0.0,Uga,UG,500,2005-12-13 12:09:18+00:00,500.0,Food Production/Sales,Food,False,Rose,Buying and reselling of produce,paid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5255,2007-03-27 05:41:31+00:00,1000,0.0,Gha,GH,1000,2008-01-10 10:17:02+00:00,1000.0,Construction,Construction,False,Grace,Working capital,paid
5256,2007-03-27 21:58:56+00:00,800,0.0,Gha,GH,800,2008-01-11 10:16:27+00:00,800.0,Food Market,Food,False,Ysutor,Working capital,paid
5257,2007-06-02 03:57:56+00:00,1000,0.0,Gha,GH,1000,2008-03-13 10:01:32+00:00,1000.0,Construction,Construction,False,Abla,Working capital,paid


## RECODIFICAR  VALORES

**map()**

Recodifica unos valores por otros según un diccionario {'valor viejo':'valor nuevo'} o una función.

Se podría hacer con replace() pero map es más flexible ya que admite funciones.

Si se usa un diccionario todo valor que no esté en el diccionario será reemplazado por un nulo.

No hace inplace.

https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html

Parámetros más importantes:

* na_action: se puede poner a 'ignore' para que no aplique la función sobre los nulos y los deje como nulos

In [55]:
#Valores originales de status
df.Status.value_counts(dropna=False)

Status
paid         4031
defaulted     600
refunded      381
deleted        86
NaN            48
Name: count, dtype: int64

In [56]:
#Ejemplo de recodificar con un diccionario
status_dict = {
    'paid':'pagado',
    'defaulted':'impagado',
    'deleted':'pagado'
}

df.Status.map(status_dict).value_counts(dropna=False)

Status
pagado      4117
impagado     600
NaN          429
Name: count, dtype: int64

In [58]:
def mayus(texto):
    return (texto.upper())

df.Status.map(mayus, na_action='ignore').head(2)

id
84    PAID
85    PAID
Name: Status, dtype: category
Categories (4, object): ['DELETED' < 'DEFAULTED' < 'PAID' < 'REFUNDED']

Repaso de lo aprendido:

- Cambia nombres de columnas o índices con rename(columns={'viejo':'nuevo'}).
- Usa replace() para cambios directos en valores específicos.
- Aplica map() para recodificaciones más flexibles con funciones o diccionarios complejos.

## MODIFICAR TEXTOS

### PRINCIPALES ACCESSORS DE TIPO TEXTO

Usaremos la variable use de df para los ejemplos. Por comodidad vamos a guardarla en una variable propia y sólo con 5 registros.

Hay muchos más de los que mostramos aquí. Te animo a que los revises al menos por alto.

In [67]:
texto = df.Use[0:5]
texto

id
84                       Buy bulls, open a butcher shop
85     Buying more produce each time for greater profit
86    Increase number of goats bought and sold each ...
88    Buy used clothing (mutumbe) of higher quality ...
89                      Buying and reselling of produce
Name: Use, dtype: object

Métodos para cambiar mayúsculas-minúsculas. Se usan mucho para estandarizar formatos, por ejemplo para aplicaciones de text mining.

In [68]:
#Todo a mayúsculas
texto.str.upper()

id
84                       BUY BULLS, OPEN A BUTCHER SHOP
85     BUYING MORE PRODUCE EACH TIME FOR GREATER PROFIT
86    INCREASE NUMBER OF GOATS BOUGHT AND SOLD EACH ...
88    BUY USED CLOTHING (MUTUMBE) OF HIGHER QUALITY ...
89                      BUYING AND RESELLING OF PRODUCE
Name: Use, dtype: object

In [69]:
#Todo a minúsculas
texto.str.lower()

id
84                       buy bulls, open a butcher shop
85     buying more produce each time for greater profit
86    increase number of goats bought and sold each ...
88    buy used clothing (mutumbe) of higher quality ...
89                      buying and reselling of produce
Name: Use, dtype: object

In [70]:
#Todo a estilo frase (primera con mayúsculas)
texto.str.capitalize()

id
84                       Buy bulls, open a butcher shop
85     Buying more produce each time for greater profit
86    Increase number of goats bought and sold each ...
88    Buy used clothing (mutumbe) of higher quality ...
89                      Buying and reselling of produce
Name: Use, dtype: object

In [71]:
#Todo a estilo titular (primera de cada palabra con mayúsculas)
texto.str.title()

id
84                       Buy Bulls, Open A Butcher Shop
85     Buying More Produce Each Time For Greater Profit
86    Increase Number Of Goats Bought And Sold Each ...
88    Buy Used Clothing (Mutumbe) Of Higher Quality ...
89                      Buying And Reselling Of Produce
Name: Use, dtype: object

Concatenar o separar textos

In [72]:
#Concatenar
texto.str.cat()

'Buy bulls, open a butcher shopBuying more produce each time for greater profitIncrease number of goats bought and sold each time for greater profitsBuy used clothing (mutumbe) of higher quality so it can be sold at a higher price for higher profitsBuying and reselling of produce'

In [73]:
#Unir cada elemento del texto por un separador
texto.str.join(sep = '--')

id
84    B--u--y-- --b--u--l--l--s--,-- --o--p--e--n-- ...
85    B--u--y--i--n--g-- --m--o--r--e-- --p--r--o--d...
86    I--n--c--r--e--a--s--e-- --n--u--m--b--e--r-- ...
88    B--u--y-- --u--s--e--d-- --c--l--o--t--h--i--n...
89    B--u--y--i--n--g-- --a--n--d-- --r--e--s--e--l...
Name: Use, dtype: object

In [74]:
#Separar
texto.str.split()

id
84                [Buy, bulls,, open, a, butcher, shop]
85    [Buying, more, produce, each, time, for, great...
86    [Increase, number, of, goats, bought, and, sol...
88    [Buy, used, clothing, (mutumbe), of, higher, q...
89                [Buying, and, reselling, of, produce]
Name: Use, dtype: object

Comprobar si contiene algún subtexto en concreto, se usa mucho para filtrar variables.

In [75]:
#Comprobar si empiezar por
filtro = df.columns.str.startswith('Funded')
df.loc[:,filtro]

,Funded Date,Funded Amount
id,,
84,2005-03-31 06:27:55+00:00,500
85,2005-03-31 06:27:55+00:00,500
86,2005-03-31 06:27:55+00:00,500
88,2005-03-31 06:27:55+00:00,300
89,2005-03-31 06:27:55+00:00,500
...,...,...
5255,2007-03-27 05:41:31+00:00,1000
5256,2007-03-27 21:58:56+00:00,800
5257,2007-06-02 03:57:56+00:00,1000


In [76]:
#Comprobar si contiene
filtro = df.columns.str.contains(('Funded'))
df.loc[:,filtro]

,Funded Date,Funded Amount
id,,
84,2005-03-31 06:27:55+00:00,500
85,2005-03-31 06:27:55+00:00,500
86,2005-03-31 06:27:55+00:00,500
88,2005-03-31 06:27:55+00:00,300
89,2005-03-31 06:27:55+00:00,500
...,...,...
5255,2007-03-27 05:41:31+00:00,1000
5256,2007-03-27 21:58:56+00:00,800
5257,2007-06-02 03:57:56+00:00,1000


In [77]:
#Comprobar si termina por
filtro = df.columns.str.endswith('Amount')
df.loc[:,filtro]

,Funded Amount,Loan Amount,Paid Amount
id,,,
84,500,500,500.0
85,500,500,500.0
86,500,500,500.0
88,300,300,300.0
89,500,500,500.0
...,...,...,...
5255,1000,1000,1000.0
5256,800,800,800.0
5257,1000,1000,1000.0


Calcular la longitud de un texto.

In [78]:
texto.str.len()

id
84     30
85     48
86     70
88    100
89     31
Name: Use, dtype: int64

In [79]:
# Reemplazar valor
df.columns.str.replace(pat = ' ', repl = '_')

Index(['Funded_Date', 'Funded_Amount', 'Pendiente', 'Country', 'Country_Code',
       'Loan_Amount', 'Paid_Date', 'Paid_Amount', 'Activity', 'Sector',
       'Delinquent', 'Name', 'Use', 'Status'],
      dtype='object')

Eliminar espacios al principio o final.

In [82]:
# Encontrar valor (devuelve la posición de la primera ocurrencia, 
# y si no lo encuentra -1)
texto.str.strip()

id
84                       Buy bulls, open a butcher shop
85     Buying more produce each time for greater profit
86    Increase number of goats bought and sold each ...
88    Buy used clothing (mutumbe) of higher quality ...
89                      Buying and reselling of produce
Name: Use, dtype: object

Repaso de lo aprendido:

- Accede a funciones avanzadas con el accessor 'str' para manipular variables tipo texto eficientemente.
- Estandariza texto con métodos como upper(), lower(), title() y capitalize().
- Utiliza métodos como contains(), startswith(), endswith() para seleccionar columnas o filas según criterios específicos.

## EJERCICIOS

Carga nuevamente el dataframe para realizar algunas correcciones

In [4]:
df_1 = pd.read_csv('../../00_Datasets//DataSetKivaCreditScoring.csv', 
                 sep = ';', index_col = 'id',
                 parse_dates = ['Funded Date','Paid Date'])

### La empresa quiere mejorar el rendimiento de su modelo, reduciendo el consumo de memoria de su base de datos de préstamos. Convierte todas las columnas de tipo object a tipo category y muestra los nuevos tipos resultantes.

In [5]:
df_1.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 5146 entries, 84 to 5259
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   Funded Date    5050 non-null   datetime64[ns, UTC]
 1   Funded Amount  5146 non-null   int64              
 2   Country        5146 non-null   object             
 3   Country Code   5146 non-null   object             
 4   Loan Amount    5146 non-null   int64              
 5   Paid Date      4121 non-null   datetime64[ns, UTC]
 6   Paid Amount    4072 non-null   float64            
 7   Activity       5146 non-null   object             
 8   Sector         5146 non-null   object             
 9   Delinquent     5146 non-null   bool               
 10  Name           5145 non-null   object             
 11  Use            5145 non-null   object             
 12  Status         5098 non-null   object             
dtypes: bool(1), datetime64[ns, UTC](2), float64(1), int6

In [7]:
col_obj = df_1.select_dtypes('object').columns
df_1[col_obj] = df_1[col_obj].astype('category')
df_1.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 5146 entries, 84 to 5259
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   Funded Date    5050 non-null   datetime64[ns, UTC]
 1   Funded Amount  5146 non-null   int64              
 2   Country        5146 non-null   category           
 3   Country Code   5146 non-null   category           
 4   Loan Amount    5146 non-null   int64              
 5   Paid Date      4121 non-null   datetime64[ns, UTC]
 6   Paid Amount    4072 non-null   float64            
 7   Activity       5146 non-null   category           
 8   Sector         5146 non-null   category           
 9   Delinquent     5146 non-null   bool               
 10  Name           5145 non-null   category           
 11  Use            5145 non-null   category           
 12  Status         5098 non-null   category           
dtypes: bool(1), category(7), datetime64[ns, UTC](2), flo

### Desde el área de analítica de la empresa se ha tomado la decición de corregir los nulos de la variable Funded Date por su moda. Identifica cuantos valores nulos hay en dicha columna y haz la imputación.

In [12]:
df_1['Funded Date'].isna().sum()

moda = df_1['Funded Date'].mode().values[0]

df_1['Funded Date'] = df_1['Funded Date'].fillna(moda)

df_1['Funded Date'].isna().sum()

np.int64(0)

### Desde el departamento de dirección necesitan conocer el número exacto de préstamos que se han hecho por el importe mínimo permitido. Posteriormente indican que si son menos de 15 pueden eliminarse dichos registros

In [26]:
minimo = df_1['Loan Amount'].min()
a_eliminar = df_1[df_1['Loan Amount'] == minimo].index
df_1.drop(a_eliminar)

,Funded Date,Funded Amount,Country,Country Code,Loan Amount,Paid Date,Paid Amount,Activity,Sector,Delinquent,Name,Use,Status
id,,,,,,,,,,,,,
84,2005-03-31 06:27:55+00:00,500,Uganda,UG,500,2005-12-13 12:00:40+00:00,500.0,Butcher Shop,Food,False,Justine,"Buy bulls, open a butcher shop",paid
85,2005-03-31 06:27:55+00:00,500,Uganda,UG,500,2005-12-13 12:04:33+00:00,500.0,Food Production/Sales,Food,False,Geoffrey,Buying more produce each time for greater profit,paid
86,2005-03-31 06:27:55+00:00,500,Uganda,UG,500,2005-12-13 12:06:56+00:00,500.0,Animal Sales,Agriculture,False,Apollo,Increase number of goats bought and sold each ...,paid
88,2005-03-31 06:27:55+00:00,300,Uganda,UG,300,2005-08-27 07:33:49+00:00,300.0,Clothing Sales,Clothing,False,Eunice,Buy used clothing (mutumbe) of higher quality ...,paid
89,2005-03-31 06:27:55+00:00,500,Uganda,UG,500,2005-12-13 12:09:18+00:00,500.0,Food Production/Sales,Food,False,Rose,Buying and reselling of produce,paid
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5255,2007-03-27 05:41:31+00:00,1000,Ghana,GH,1000,2008-01-10 10:17:02+00:00,1000.0,Construction,Construction,False,Grace,Working capital,paid
5256,2007-03-27 21:58:56+00:00,800,Ghana,GH,800,2008-01-11 10:16:27+00:00,800.0,Food Market,Food,False,Ysutor,Working capital,paid
5257,2007-06-02 03:57:56+00:00,1000,Ghana,GH,1000,2008-03-13 10:01:32+00:00,1000.0,Construction,Construction,False,Abla,Working capital,paid


### Nos solicitan la revisión de la variable nombres del dataset, porque necesitan hacer un envío de información. Revisa sus valores, si hay nulos u otros valores que puedan afectar al envío masivo, crea un nuevo dataset limpio y listo para este objetivo.

In [31]:
df_1[df_1.Name.isna()]
df_1.Name.value_counts()

Name
Anonymous    983
Maria         48
Mary          32
Grace         26
Esther        24
            ... 
Zemfira        1
Zarina         1
Zara           1
Zanfira        1
Zamina         1
Name: count, Length: 2633, dtype: int64

In [35]:
df_mail = df_1.dropna(subset='Name')
df_mail = df_mail[df_mail.Name != 'Anonymous']

In [37]:
df_mail.Name.value_counts()

Name
Maria        48
Mary         32
Grace        26
Esther       24
Rosa         22
             ..
Zemfira       1
Zarina        1
Zara          1
Zanfira       1
Anonymous     0
Name: count, Length: 2633, dtype: int64

In [40]:
df_mail.Name = df_mail.Name.astype('str')
df_mail.Name.value_counts()

Name
Maria      48
Mary       32
Grace      26
Esther     24
Rosa       22
           ..
Sisili      1
Samaya      1
Rasela      1
Ysutor      1
Tunzala     1
Name: count, Length: 2632, dtype: int64

### El equipo de BI nos pide que estandaricemos los nombres de las variables y así poder tenerlas ellos disponibles en el formato habitual: mayúsculas y guiones en vez de espacios.

In [44]:
df_1.columns = df_1.columns.str.replace(' ','_').str.upper()
df_1.head(2)

,FUNDED_DATE,FUNDED_AMOUNT,COUNTRY,COUNTRY_CODE,LOAN_AMOUNT,PAID_DATE,PAID_AMOUNT,ACTIVITY,SECTOR,DELINQUENT,NAME,USE,STATUS
id,,,,,,,,,,,,,
84,2005-03-31 06:27:55+00:00,500,Uganda,UG,500,2005-12-13 12:00:40+00:00,500.0,Butcher Shop,Food,False,Justine,"Buy bulls, open a butcher shop",paid
85,2005-03-31 06:27:55+00:00,500,Uganda,UG,500,2005-12-13 12:04:33+00:00,500.0,Food Production/Sales,Food,False,Geoffrey,Buying more produce each time for greater profit,paid


### El equipo de analistas quiere centrarse solo en las columnas que contienen la palabra Amount en su nombre. Extrae esas columnas y muestra las primeras 5 filas.

In [47]:
filtro = df_1.columns.str.contains('AMOUNT')
df_1.loc[:, filtro].head(5)

,FUNDED_AMOUNT,LOAN_AMOUNT,PAID_AMOUNT
id,,,
84,500,500,500.0
85,500,500,500.0
86,500,500,500.0
88,300,300,300.0
89,500,500,500.0
